# 列表生成式

## 列表生成式的理解方式

 列表表达式类似于数学中的集合<br>
 for 表示值域<br>
 if 表示约束<br>
 最后的结果就是集合中是所有元素

In [3]:
even = {
    x 
    for x in range(10)
    if x%2 == 0
}
even

{0, 2, 4, 6, 8}

even等价于 {x|x∈(0,1,2,3,4,5,6,7,8,9),x%2=0} <br>
所以列表生成式的本质就是集合

## 列表生成式的执行顺序

列表生成式结构:<br>
(结果表达式 for|if 表达式)<br><br>
列表生成式的顺序: <br>
for优先级=if优先级>结果表达式<br>
优先级相同执行顺序为从左到右，从上到下

In [25]:
def echo_range(iter_type):
    for i in iter_type:
        print('execute range', i)
        yield i

def echo_if(data):
    print('execute if', data)
    return data

def echo_data(data):
    print('execute data', data)
    return data

### part1

In [26]:
first_range = echo_range(range(5))
data = [
    echo_data(i)
    for i in first_range
    if echo_if(i) > 2
]
data 

execute range 0
execute if 0
execute range 1
execute if 1
execute range 2
execute if 2
execute range 3
execute if 3
execute data 3
execute range 4
execute if 4
execute data 4


[3, 4]

测试后可以看到先执行for循环，再执行if判断，当if判断通过后，才会进行结果表达式的计算<br>
接下来进行一些再复杂一些的测试

### part2

In [29]:
first_range = echo_range(range(3))
second_range = lambda:echo_range(['a','b','c'])  # 每次创建一个新的迭代器,否则迭代器会被耗尽
data  = [
    echo_data(i)
    for i in first_range
    if echo_if(i)%2 == 0
    for j in second_range()
    if echo_if(j) == 'a'
]
data

execute range 0
execute if 0
execute range a
execute if a
execute data 0
execute range b
execute if b
execute range c
execute if c
execute range 1
execute if 1
execute range 2
execute if 2
execute range a
execute if a
execute data 2
execute range b
execute if b
execute range c
execute if c


[0, 2]

上面我尝试写了更多的for和if，它们依然遵守从左到右，从上到下的执行顺序<br>
而在if后面的表达式，在if判断为真时才会向下执行<br>

这里有一个比较有意思的地方，那就是我的second_range是一个匿名函数，它的功能就是返回一个新的生成器<br>
如果不这样做，那么在第一次if通过的时候，second_range就会被耗尽，这会导致当i为2时，虽然if判断通过了，但是因为second_range被耗尽了，无法再向下循环，所以i为2的值不会出现在最后结果里<br>
以下代码展示了这个问题

In [31]:
first_range = echo_range(range(3))
second_range = echo_range(['a','b','c'])
data  = [
    echo_data(i)
    for i in first_range
    if echo_if(i)%2 == 0
    for j in second_range
    if echo_if(j) == 'a'
]
data

execute range 0
execute if 0
execute range a
execute if a
execute data 0
execute range b
execute if b
execute range c
execute if c
execute range 1
execute if 1
execute range 2
execute if 2


[0]

### part3

In [40]:
raw = {
    'a':['b','c'],
    'd':['e','f']
}
data = [
    {echo_data(i):echo_data(k) for i in echo_range(v)}
    for k,v in echo_range(raw.items())
]
data

execute range ('a', ['b', 'c'])
execute range b
execute data b
execute data a
execute range c
execute data c
execute data a
execute range ('d', ['e', 'f'])
execute range e
execute data e
execute data d
execute range f
execute data f
execute data d


[{'b': 'a', 'c': 'a'}, {'e': 'd', 'f': 'd'}]

在新的代码里，我在结果表达式中又嵌套了一层for循环<br>
看似复杂，实际上它仍然按照上文的规则来运行<br>
先是对raw的键值对进行遍历<br>
然后是执行结果表达式<br>
于是进行第二次循环，对原来为value的列表进行遍历，作为新的字典的key来使用<br>
而原来key则作为value<br>
第二次循环结束后，生成新的字典，结果表达式结束
回到第一次循环，取下一个值重复上述动作